# Example of using LLM-based pipeline for recommendation task

Run in google colaboratory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
import os

repo_path = '/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec'
sys.path.append(repo_path)

In [4]:
%cd '{repo_path}'

/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec


## Install requirements

In [3]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

## Get dataset and config

In [6]:
from llm4rec.pipelines import RecBolePipelineRecommender
from llm4rec.dataset import RecboleSeqDataset
from llm4rec.evaluation.trainer import PipelineTrainer
from llm4rec.utils.dataset_utils import ml100k_preprocess
from recbole.data.utils import data_preparation
from recbole.config import Config
import os
import torch


model_cls = RecBolePipelineRecommender
dataset_name = 'ml-100k'

config = Config(model=model_cls, dataset=dataset_name,
             config_file_list=['./llm4rec/configs/dataset_ml100k.yaml',
                               './llm4rec/configs/overall.yaml'])

dataset = RecboleSeqDataset(config, preprocess_text_fn=ml100k_preprocess)
_, _, test_data = data_preparation(config, dataset)


## Create open source information retrieval using HuggingFace Sentence Transformers model and open source Ranker using Mistral-7b

In [9]:
from llm4rec.tasks import RetrievalRecommender

retrieval = RetrievalRecommender(
                embeddings=None,
                item2text=dataset.item_token2text,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])})

In [10]:
from langchain import HuggingFaceHub
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from llm4rec.tasks import RankerRecommender
import os

path_to_env = os.path.join(repo_path, "api_keys.env")
load_dotenv(path_to_env)

llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)
#llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2", model_kwargs={"temperature":0.1, "max_length":512})
ranker = RankerRecommender(llm=llm, item2text=dataset.item_token2text)

## Evaluate pipeline

In [12]:
model = RecBolePipelineRecommender(config=config,
                                   dataset=dataset,
                                tasks=[retrieval, ranker], verbose=False)

trainer = PipelineTrainer(config, model)
test_result = trainer.evaluate(test_data, show_progress=config['show_progress'])

Evaluate   :   0%|                                                          | 0/118 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Evaluate   :   1%|▍                                                 | 1/118 [00:27<52:42, 27.03s/it]


KeyboardInterrupt: 

In [ ]:
test_result

OrderedDict([('recall@1', 0.0),
             ('recall@5', 0.0),
             ('recall@20', 0.0208),
             ('ndcg@1', 0.0),
             ('ndcg@5', 0.0),
             ('ndcg@20', 0.0058)])